In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
# Do you wanna see?
verbose = True

from utils.data_extraction import load_data
from utils.data_cleaning import HFDataCleaning

### Load data etc

In [2]:
# print(os.listdir())
path = 'T:/taqhdf5'  #'a:/taqhdf5'
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,32)]).astype(int)
#dates = np.array(['20200401']).astype(int)

# Provide a list of tickers of interest
tickers = ['GOOG']#'MSFT'

# Do we need data on trades, quotes or both?
dataNeeded = 'trades' # 'trades', 'quotes' or 'both'

# run load_data()
if dataNeeded == 'trades':
    tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'quotes':
    quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
elif dataNeeded == 'both':
    tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

['.git', '.gitignore', '.ipynb_checkpoints', 'CrunchTAQ.ipynb', 'hello.py', 'README.md', 'Speciale to-do.docx', 'Speciale to-do.txt', 'test', 'test_and_compare_candlesticks.ipynb', 'utils']


FileNotFoundError: [WinError 3] Den angivne sti blev ikke fundet: 'T:/taqhdf5'

In [5]:
cleanedData = HFDataCleaning(['P1','p2','t1','p3'],tradeData,'trade',['q'])

In [6]:
step = 10 # in minutes

aggregateMinute = np.arange(0,60,step)
aggregateHour = np.arange(9,16,1)
aggregateDate = np.arange(len(cleanedData.Date.unique()))

remove = 30//step

# candle = np.zeros(((len(aggregateDate)*len(aggregateMinute)*len(aggregateHour)),4))
candleNP = np.zeros((((len(aggregateDate)*len(aggregateMinute)*len(aggregateHour))-remove*len(aggregateDate)),4))

In [7]:
numpiedData = cleanedData[['Date','Hour','Minute']].to_numpy()
numpiedData = numpiedData.T
numpiedPrice = cleanedData['price'].to_numpy()

### Original

In [8]:


def candleCreateNP():
    ii = 0
    for l in cleanedData.Date.unique():
        for i in aggregateHour:
            for j in aggregateMinute:
                if (i == 9) & (j <30):
                    continue
                
                p1 = numpiedPrice[((numpiedData[0]==l)&\
                                     (numpiedData[1]==i)&\
                                     (numpiedData[2]>=j))&((numpiedData[0]==l)&\
                                                           (numpiedData[1]==i)&\
                                                           (numpiedData[2]<j+step))]
                if len(p1) > 0:
                    candleNP[ii] = np.array([p1[0],p1.max(),p1.min(),p1[-1]])
                else:
                    # if no new prices in the interval considered, use the previous pne
                    candleNP[ii] = candleNP[ii-1]
                ii += 1
                
    return candleNP

In [9]:
# Aggregate data in candle sticks
candleNP = candleCreateNP()

candleNP.shape

(39, 4)

In [10]:
candleNP[:5], candleNP[-5:]

(array([[1122.26, 1128.7 , 1114.79, 1124.52],
        [1124.72, 1127.85, 1118.26, 1120.29],
        [1120.13, 1121.68, 1115.18, 1115.55],
        [1115.35, 1126.  , 1114.3 , 1124.3 ],
        [1124.33, 1129.31, 1123.  , 1124.66]]),
 array([[1105.4 , 1105.71, 1101.8 , 1104.  ],
        [1104.05, 1104.05, 1100.36, 1100.69],
        [1100.76, 1101.28, 1097.44, 1097.88],
        [1097.5 , 1110.68, 1097.21, 1106.87],
        [1106.62, 1110.97, 1102.43, 1105.62]]))

### Attempt to vectorize

In [11]:
cleanedData['hour_min_col'] = cleanedData['Hour'] + cleanedData['Minute']/60
cleanedData.head()

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,ParticipantTime,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col
0,34200012706854,Q,@ TI,83,1122.26,,00,36579,447,N,...,34200012683939,99,0,20200401,2020-04-01 09:30:00.012706,854,9,30,GOOG,9.5
1,34200012711048,Q,@ TI,17,1122.26,,00,36580,448,N,...,34200012683939,99,0,20200401,2020-04-01 09:30:00.012711,048,9,30,GOOG,9.5
2,34200012771200,Q,@ TI,83,1122.26,,00,36583,449,N,...,34200012747399,99,0,20200401,2020-04-01 09:30:00.012771,200,9,30,GOOG,9.5
3,34200012827903,Q,@ TI,83,1122.26,,00,36584,450,N,...,34200012803083,99,0,20200401,2020-04-01 09:30:00.012827,903,9,30,GOOG,9.5
4,34200013120215,Q,@ TI,20,1122.08,,00,36589,451,N,...,34200013098975,99,0,20200401,2020-04-01 09:30:00.013120,215,9,30,GOOG,9.5


In [12]:
cleanedData['hour_min_col'].min(), cleanedData['hour_min_col'].max()

(9.5, 15.983333333333333)

In [13]:
time_bins = np.arange(9.5,16+step/60,step/60)
time_bins

array([ 9.5       ,  9.66666667,  9.83333333, 10.        , 10.16666667,
       10.33333333, 10.5       , 10.66666667, 10.83333333, 11.        ,
       11.16666667, 11.33333333, 11.5       , 11.66666667, 11.83333333,
       12.        , 12.16666667, 12.33333333, 12.5       , 12.66666667,
       12.83333333, 13.        , 13.16666667, 13.33333333, 13.5       ,
       13.66666667, 13.83333333, 14.        , 14.16666667, 14.33333333,
       14.5       , 14.66666667, 14.83333333, 15.        , 15.16666667,
       15.33333333, 15.5       , 15.66666667, 15.83333333, 16.        ,
       16.16666667])

In [14]:
cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=False, labels=False) #0#+1
cleanedData

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col,time_group
0,34200012706854,Q,@ TI,83,1122.26,,00,36579,447,N,...,99,0,20200401,2020-04-01 09:30:00.012706,854,9,30,GOOG,9.500000,0
1,34200012711048,Q,@ TI,17,1122.26,,00,36580,448,N,...,99,0,20200401,2020-04-01 09:30:00.012711,048,9,30,GOOG,9.500000,0
2,34200012771200,Q,@ TI,83,1122.26,,00,36583,449,N,...,99,0,20200401,2020-04-01 09:30:00.012771,200,9,30,GOOG,9.500000,0
3,34200012827903,Q,@ TI,83,1122.26,,00,36584,450,N,...,99,0,20200401,2020-04-01 09:30:00.012827,903,9,30,GOOG,9.500000,0
4,34200013120215,Q,@ TI,20,1122.08,,00,36589,451,N,...,99,0,20200401,2020-04-01 09:30:00.013120,215,9,30,GOOG,9.500000,0
5,34200134121465,Q,@ TI,1,1122.25,,00,36816,452,N,...,99,0,20200401,2020-04-01 09:30:00.134121,465,9,30,GOOG,9.500000,0
6,34200137711405,Q,@ TI,1,1121.00,,00,36821,453,N,...,99,0,20200401,2020-04-01 09:30:00.137711,405,9,30,GOOG,9.500000,0
7,34200194838863,Q,@FTI,38,1122.26,,00,36870,454,N,...,99,1,20200401,2020-04-01 09:30:00.194838,863,9,30,GOOG,9.500000,0
8,34200194842233,Q,@FTI,1,1122.26,,00,36871,455,N,...,99,1,20200401,2020-04-01 09:30:00.194842,233,9,30,GOOG,9.500000,0
9,34200194877651,Q,@FTI,15,1124.00,,00,36872,456,N,...,99,1,20200401,2020-04-01 09:30:00.194877,651,9,30,GOOG,9.500000,0


In [15]:
cleanedData[(cleanedData['hour_min_col'] > 9.6) & (cleanedData['hour_min_col'] < 9.7)]

,utcsec,ex,cond,volume,price,TradeStopStockIndicator,corr,TradeSequenceNumber,TradeID,SourceOfTrade,...,TRFTime,TTE,Date,Timestamp,TSRemainder,Hour,Minute,Ticker,hour_min_col,time_group
1774,34621200075630,Q,@F I,6,1124.98,,00,157628,2221,N,...,99,1,20200401,2020-04-01 09:37:01.200075,630,9,37,GOOG,9.616667,0
1775,34621200080193,Q,@F I,10,1125.00,,00,157629,2222,N,...,99,1,20200401,2020-04-01 09:37:01.200080,193,9,37,GOOG,9.616667,0
1776,34621200192318,Q,@F I,5,1125.00,,00,157630,2223,N,...,99,1,20200401,2020-04-01 09:37:01.200192,318,9,37,GOOG,9.616667,0
1777,34621203396369,Q,@F I,13,1125.52,,00,157653,2224,N,...,99,1,20200401,2020-04-01 09:37:01.203396,369,9,37,GOOG,9.616667,0
1778,34621204241407,Q,@F I,7,1125.52,,00,157655,2225,N,...,99,1,20200401,2020-04-01 09:37:01.204241,407,9,37,GOOG,9.616667,0
1779,34624846143673,Q,@F I,20,1126.00,,00,158379,2226,N,...,99,1,20200401,2020-04-01 09:37:04.846143,673,9,37,GOOG,9.616667,0
1780,34624846147812,Q,@F I,1,1126.00,,00,158380,2227,N,...,99,1,20200401,2020-04-01 09:37:04.846147,812,9,37,GOOG,9.616667,0
1781,34624847803554,Q,@F I,1,1126.41,,00,158400,2228,N,...,99,1,20200401,2020-04-01 09:37:04.847803,554,9,37,GOOG,9.616667,0
1782,34624849420780,Q,@F I,9,1126.50,,00,158413,2229,N,...,99,1,20200401,2020-04-01 09:37:04.849420,780,9,37,GOOG,9.616667,0
1783,34624849670690,Q,@ I,14,1126.50,,00,158414,2230,N,...,99,0,20200401,2020-04-01 09:37:04.849670,690,9,37,GOOG,9.616667,0


In [16]:
cleanedData.groupby(['Date','time_group'])[['price','Timestamp']].first() # .last()

price                  Timestamp
Date     time_group                                    
20200401 0           1122.26 2020-04-01 09:30:00.012706
         1           1124.72 2020-04-01 09:40:02.376757
         2           1120.13 2020-04-01 09:50:00.514164
         3           1115.35 2020-04-01 10:00:00.001645
         4           1124.33 2020-04-01 10:10:03.030533
         5           1124.81 2020-04-01 10:20:10.697902
         6           1122.58 2020-04-01 10:30:03.432125
         7           1127.67 2020-04-01 10:40:04.582640
         8           1125.80 2020-04-01 10:50:05.461097
         9           1127.00 2020-04-01 11:00:10.846559
         10          1128.12 2020-04-01 11:10:01.552534
         11          1126.27 2020-04-01 11:20:06.754553
         12          1121.14 2020-04-01 11:30:02.422136
         13          1119.04 2020-04-01 11:40:00.895746
         14          1117.15 2020-04-01 11:50:09.753210
         15          1117.68 2020-04-01 12:00:01.142005
         16          1115.74 2020-04-01 12:10:05.074493
         17          1114.14 2020-04-01 12:20:07.501602
         18          1111.49 2020-04-01 12:30:04.166204
         19          1106.45 2020-04-01 12:40:05.897464
         20          1105.54 2020-04-01 12:50:12.640710
         21          1108.44 2020-04-01 13:00:00.130828
         22          1110.52 2020-04-01 13:10:06.700589
         23          1110.99 2020-04-01 13:20:13.416221
         24          1111.00 2020-04-01 13:30:12.778582
         25          1111.65 2020-04-01 13:40:12.401546
         26          1108.00 2020-04-01 13:50:08.091597
         27          1111.77 2020-04-01 14:00:00.018328
         28          1111.88 2020-04-01 14:10:00.620321
         29          1109.05 2020-04-01 14:20:00.765345
         30          1108.54 2020-04-01 14:30:00.382845
         31          1109.73 2020-04-01 14:40:00.265699
         32          1109.00 2020-04-01 14:50:19.920914
         33          1105.10 2020-04-01 15:00:00.516835
         34          1105.40 2020-04-01 15:10:15.900661
         35          1104.05 2020-04-01 15:20:02.029433
         36          1100.76 2020-04-01 15:30:03.342836
         37          1097.50 2020-04-01 15:40:02.460831
         38          1106.62 2020-04-01 15:50:00.019019

In [17]:
OHLC = cleanedData.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])
OHLC.values.shape

(39, 4)

In [18]:
OHLC.values[:5], OHLC.values[-5:]

(array([[1122.26, 1128.7 , 1114.79, 1124.52],
        [1124.72, 1127.85, 1118.26, 1120.29],
        [1120.13, 1121.68, 1115.18, 1115.55],
        [1115.35, 1126.  , 1114.3 , 1124.3 ],
        [1124.33, 1129.31, 1123.  , 1124.66]]),
 array([[1105.4 , 1105.71, 1101.8 , 1104.  ],
        [1104.05, 1104.05, 1100.36, 1100.69],
        [1100.76, 1101.28, 1097.44, 1097.88],
        [1097.5 , 1110.68, 1097.21, 1106.87],
        [1106.62, 1110.97, 1102.43, 1105.62]]))

In [19]:
# set up as function
def candleCreateNP_vect(verbose=True):
        
    cleanedData['hour_min_col'] = cleanedData['Hour'] + cleanedData['Minute']/60
    if verbose:
        print(f"min and max of new hour_min_col: \
              {cleanedData['hour_min_col'].min()}, {cleanedData['hour_min_col'].max()}")
              
    # setup time_bins to group each timestamp
    time_bins = np.arange(9.5, 16+step/60, step/60)
              
    # put each timestamp into a bucket according to time_bins defined by the step variable
    cleanedData['time_group'] = pd.cut(cleanedData['hour_min_col'], bins=time_bins, right=False, labels=False)
    
    # group by date and time_group, extract price, take it first, max, min, last (open, high, low, close)
    OHLC = cleanedData.groupby(['Date','time_group'])[['price']].agg(['first', 'max', 'min', 'last'])              
    
    # return as numpy if preferred
    return OHLC.values

### Compare timing once

In [20]:
import time
a = time.time()
candleNP1 = candleCreateNP()
b = time.time()
print(b-a)

a = time.time()
candleNP2 = candleCreateNP_vect()
b = time.time()
print(b-a)

0.15100860595703125
min and max of new hour_min_col:               9.5, 15.983333333333333
0.015001058578491211


In [21]:
# Make sure they are equal
(candleNP1.shape == candleNP2.shape), \
(candleNP1 == candleNP2).all(), \
np.array_equal(candleNP1, candleNP2)

(True, True, True)

### Multiple time-it runs

In [28]:
# Aggregate data in candle sticks
%timeit candleNP1 = candleCreateNP()
%timeit candleNP2 = candleCreateNP_vect(verbose=False)

118 ms ± 584 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
13.8 ms ± 21.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
